In [22]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


raw = DataFrame()
raw['Feature1'] = [x for x in range(10)]
raw['Feature2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values,n_in=7)
#data = series_to_supervised(values,n_in=7).values.reshape(-1,7,2)
print(data)

   var1(t-7)  var2(t-7)  var1(t-6)  var2(t-6)  var1(t-5)  var2(t-5)  \
7        0.0       50.0        1.0       51.0        2.0       52.0   
8        1.0       51.0        2.0       52.0        3.0       53.0   
9        2.0       52.0        3.0       53.0        4.0       54.0   

   var1(t-4)  var2(t-4)  var1(t-3)  var2(t-3)  var1(t-2)  var2(t-2)  \
7        3.0       53.0        4.0       54.0        5.0       55.0   
8        4.0       54.0        5.0       55.0        6.0       56.0   
9        5.0       55.0        6.0       56.0        7.0       57.0   

   var1(t-1)  var2(t-1)  var1(t)  var2(t)  
7        6.0       56.0        7       57  
8        7.0       57.0        8       58  
9        8.0       58.0        9       59  
